In [1]:
import pandas as pd
import numpy as np
import ast
from tabulate import tabulate
import json
import yaml
import matplotlib.pyplot as plt
import itertools
import dataframe_image as dfi


pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [2]:
filename = "REALRUN"  ## insert the name of the file used in the runner.py
df = pd.read_csv(rf"../output/{filename}.csv")

In [3]:
# List of metric names
metric_names = [
    "lift",
    "f1",
    "mse",
    "accuracy",
    "recall",
    "precision",
    "roc_auc",
    "area_under_pr",
    "r2_score",
    "rmse",
]
dataset = df.copy()
# Create columns based on metric names
# Convert string representations to dictionaries
dataset["output_metrics"] = dataset["output_metrics"].apply(ast.literal_eval)
dataset["run_time"] = dataset["run_time"].apply(lambda x: np.round(x / 60, 2))
# Create columns based on metric names
for metric in metric_names:
    dataset[metric] = dataset["output_metrics"].apply(
        lambda x: x.get(metric) if isinstance(x, dict) else np.nan
    )
    dataset[f"{metric}_std"] = dataset[metric].apply(
        lambda x: np.std(x) if isinstance(x, list) else np.nan
    )
    dataset[metric] = dataset[metric].apply(
        lambda x: np.average(x) if isinstance(x, list) else np.nan
    )

base_cols = [
    "dataset",
    "model",
    "run_time",
    "eval_metric",
    "best_score",
    "score_std",
    "output_metrics",
]

base_cols += [i for i in metric_names]
base_cols += [i + "_std" for i in metric_names]

dataset = dataset[base_cols]

In [4]:
tomax = {
    "mse": False,
    "rmse": False,
    "accuracy": True,
    "recall": True,
    "precision": True,
    "roc_auc": True,
    "area_under_pr": True,
    "lift": True,
    "f1": True,
    "r2_score": True,
}


# Define a function to select the first row based on whether to maximize or minimize the "best_score"
def select_first_row(group):
    metric = group["eval_metric"].iloc[0]
    ascending = not tomax.get(
        metric, True
    )  # If metric not in tomax, assume True (maximize)
    return group.sort_values(by="best_score", ascending=ascending).iloc[0]


# Apply the function to each group
filtered_df = dataset.groupby(["dataset", "model", "eval_metric"]).apply(
    select_first_row
)
# Reset the index to get a new DataFrame
filtered_df = filtered_df.reset_index(drop=True)
# Find the rows that maximize the specified metric for each dataset


# Sort the DataFrame based on whether the metric is to be maximized or not
filtered_df["ascending"] = filtered_df["eval_metric"].map(
    {k: not v for k, v in tomax.items()}
)  # Create a new column for ascending order
dfmax = filtered_df.loc[filtered_df["ascending"] == False].sort_values(
    by=["dataset", "eval_metric", "best_score"], ascending=[False, False, False]
)
dfmin = filtered_df.loc[filtered_df["ascending"] == True].sort_values(
    by=["dataset", "eval_metric", "best_score"], ascending=[False, False, True]
)

best_df = pd.concat([dfmax, dfmin])
best_df.drop(columns=["ascending"], inplace=True)

best_df.head(1)

,dataset,model,run_time,eval_metric,best_score,score_std,output_metrics,lift,f1,mse,accuracy,recall,precision,roc_auc,area_under_pr,r2_score,rmse,lift_std,f1_std,mse_std,accuracy_std,recall_std,precision_std,roc_auc_std,area_under_pr_std,r2_score_std,rmse_std
58,titanic,xgb,17.53,roc_auc,0.885801,0.027473,"{'recall': [0.7536231884057971, 0.720588235294...",2.48291,0.770295,NaN,0.836106,0.719096,0.830398,0.885801,0.864007,NaN,NaN,0.119197,0.044381,NaN,0.028001,0.052218,0.038224,0.027473,0.028585,NaN,NaN


# Check State of Run

In [5]:
with open("../configuration/experiment_config.yml", "r") as f:
    config = yaml.safe_load(f)

# Extract the necessary information from the configuration file
included_models = [i.lower() for i in config["include_models"]]
included_datasets = [i.lower() for i in config["include_datasets"]]

# Get all combinations of items from the two lists
combinations_possible = list(itertools.product(included_datasets, included_models))


# Get unique combinations based on 'Column1' and 'Column2'
existing_combinations = df[["dataset", "model"]].drop_duplicates()
# Convert the DataFrame to a list of tuples
existing_combinations = [tuple(row) for row in existing_combinations.values]

missing_combos = [i for i in combinations_possible if i not in existing_combinations]
for i in missing_combos:
    print(f"Missing Combination {i}")

Missing Combination ('iris', 'catboost')
Missing Combination ('iris', 'resnet')
Missing Combination ('titanic', 'catboost')
Missing Combination ('titanic', 'resnet')
Missing Combination ('breastcancer', 'catboost')
Missing Combination ('breastcancer', 'tabnet')
Missing Combination ('breastcancer', 'resnet')
Missing Combination ('ageconditions', 'catboost')
Missing Combination ('ageconditions', 'tabnet')
Missing Combination ('ageconditions', 'resnet')
Missing Combination ('diabetes', 'catboost')
Missing Combination ('diabetes', 'mlp')
Missing Combination ('diabetes', 'tabnet')
Missing Combination ('diabetes', 'gate')
Missing Combination ('diabetes', 'resnet')
Missing Combination ('diabetes', 's1dcnn')
Missing Combination ('diabetes', 'fttransformer')
Missing Combination ('diabetes', 'tabtransformer')
Missing Combination ('diabetes', 'gandalf')
Missing Combination ('adult', 'catboost')
Missing Combination ('adult', 'mlp')
Missing Combination ('adult', 'tabnet')
Missing Combination ('adul

In [6]:
[i for i in missing_combos if "xgb" in i]

[('covertype', 'xgb')]

In [7]:
[i for i in missing_combos if "node" in i]

[('creditcard', 'node'), ('covertype', 'node')]

In [25]:
best_df["dataset"].value_counts()

dataset
titanic          11
iris             11
breastcancer     10
ageconditions    10
diabetes          4
creditcard        4
housing           3
heloc             3
adult             2
covertype         1
Name: count, dtype: int64

In [9]:
df.loc[df["model"] == "gate"]["best_params"].iloc[2]

"{'AdamW_weight_decay': 2.084850753684019e-05, 'Adam_weight_decay': 9.742659197216048e-05, 'ExponentialLR_gamma': 0.9413668606403313, 'ReduceLROnPlateau_factor': 0.12672135640291163, 'ReduceLROnPlateau_patience': 4, 'batch_size': 53, 'chain_trees': False, 'embedding_dropout': 0, 'gflu_dropout': 0.02962414063792255, 'gflu_stages': 6, 'num_trees': 9, 'optimizer_fn': <class 'torch.optim.adam.Adam'>, 'scheduler_fn': <class 'torch.optim.lr_scheduler.ReduceLROnPlateau'>, 'tree_depth': 4, 'tree_dropout': 0.03626895389054083, 'tree_wise_attention_dropout': 0.04912261495844525, 'outer_params': {'hyperopt_evals': 50, 'auto_lr_find': True, 'max_epochs': 1000, 'val_size': 0.15, 'early_stopping_patience': 5}}"

# Execution Time Visualization

In [10]:
best_df.sort_values("run_time", ascending=False).head(20)

,dataset,model,run_time,eval_metric,best_score,score_std,output_metrics,lift,f1,mse,accuracy,recall,precision,roc_auc,area_under_pr,r2_score,rmse,lift_std,f1_std,mse_std,accuracy_std,recall_std,precision_std,roc_auc_std,area_under_pr_std,r2_score_std,rmse_std
22,covertype,categoryembedding,1263.83,accuracy,0.863411,0.009302,"{'accuracy': [0.8540113517072042, 0.8590441621...",NaN,0.867012,NaN,0.863411,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.008665,NaN,0.009302,NaN,NaN,NaN,NaN,NaN,NaN
23,creditcard,autoint,952.01,lift,9.553110,0.189117,"{'recall': [0.8686868686868687, 0.939393939393...",9.553110,0.110934,NaN,0.972199,0.908555,0.059508,0.985149,0.743908,NaN,NaN,0.189117,0.032113,NaN,0.009220,0.034552,0.018546,0.005591,0.017320,NaN,NaN
52,titanic,gate,542.06,roc_auc,0.868173,0.027653,"{'recall': [0.8115942028985508, 0.764705882352...",2.482910,0.755746,NaN,0.811431,0.762873,0.749306,0.868173,0.839969,NaN,NaN,0.068732,0.031849,NaN,0.020119,0.046156,0.018810,0.027653,0.030412,NaN,NaN
25,creditcard,tabtransformer,529.36,lift,8.981260,0.799423,"{'recall': [0.898989898989899, 0.9292929292929...",8.981260,0.074895,NaN,0.754664,0.892187,0.040459,0.933004,0.618145,NaN,NaN,0.799423,0.067703,NaN,0.268092,0.034706,0.037574,0.063408,0.066437,NaN,NaN
24,creditcard,categoryembedding,501.36,lift,9.695353,0.128761,"{'recall': [0.9292929292929293, 0.949494949494...",9.695353,0.098446,NaN,0.968217,0.920656,0.052218,0.986143,0.736194,NaN,NaN,0.128761,0.025542,NaN,0.010511,0.023800,0.014207,0.006113,0.043292,NaN,NaN
6,ageconditions,gate,458.64,f1,0.723264,0.109557,"{'recall': [0.8181818181818182, 0.818181818181...",4.825036,0.723264,NaN,0.897600,0.768398,0.693804,0.926048,0.806264,NaN,NaN,0.447543,0.109557,NaN,0.043642,0.130460,0.119922,0.047642,0.083648,NaN,NaN
27,diabetes,autoint,384.41,lift,2.394513,0.103216,"{'recall': [0.5704225352112676, 0.624394539850...",2.394513,0.271362,NaN,0.635576,0.607999,0.174736,0.675428,0.217688,NaN,NaN,0.103216,0.004744,NaN,0.013606,0.022473,0.003647,0.007759,0.011717,NaN,NaN
16,breastcancer,gate,353.53,f1,0.991686,0.008053,"{'recall': [1.0, 1.0, 0.9861111111111112, 1.0,...",1.593897,0.991686,NaN,0.989458,0.997222,0.986263,0.996230,0.997266,NaN,NaN,0.010042,0.008053,NaN,0.010227,0.005556,0.012253,0.007540,0.005467,NaN,NaN
41,iris,gate,319.40,accuracy,0.966667,0.042164,"{'accuracy': [1.0, 1.0, 0.9333333333333333, 1....",NaN,0.966617,NaN,0.966667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.042243,NaN,0.042164,NaN,NaN,NaN,NaN,NaN,NaN
29,diabetes,node,303.69,lift,2.427092,0.072793,"{'recall': [0.5919894366197183, 0.626155878467...",2.427092,0.272623,NaN,0.636411,0.609665,0.175882,0.677912,0.219922,NaN,NaN,0.072793,0.006854,NaN,0.027978,0.035736,0.007003,0.005986,0.008984,NaN,NaN


In [11]:
# Group by the "dataset" column and aggregate "run_time" using the sum function
hyperopt_evals = 1
num_parallel = 2
efficiency_estimate = 0.8
aggregated_df = best_df.groupby("model")["run_time"].sum().reset_index()
# Rename the aggregated column for clarity
aggregated_df = aggregated_df.rename(columns={"run_time": "sum_run_time"})
aggregated_df["total_search_hours_estimate"] = (
    aggregated_df["sum_run_time"] * hyperopt_evals / 60 / num_parallel * 0.8
)
aggregated_df.sort_values("total_search_hours_estimate", ascending=False).head(20)

,model,sum_run_time,total_search_hours_estimate
1,categoryembedding,2291.27,15.275133
4,gate,1673.63,11.157533
0,autoint,1411.00,9.406667
9,tabtransformer,711.17,4.741133
6,node,675.51,4.503400
7,s1dcnn,640.56,4.270400
10,xgb,473.41,3.156067
2,fttransformer,231.34,1.542267
3,gandalf,147.30,0.982000
8,tabnet,140.03,0.933533


In [12]:
# Convert DataFrame to ASCII table
class Format:
    end = "\033[0m"
    underline = "\033[4m"


def make_results_table(
    df, dataset_name, display_cols, metric_cols, image_name="", dpi=1200, image_path="/home/boom/sdev/WTabRun/notebooks/tables/",
    image_folder = ""
):
    result_df = df.loc[df["dataset"] == dataset_name].reset_index(drop=True).copy()

    # Create a dictionary to store the indices of the rows with the highest values for each metric column
    max_indices = {}
    for metric in metric_cols:
        max_indices[metric] = result_df[metric].idxmax()

    # Modify all columns with std to include relative std
    result_df["best_score"] = result_df.apply(
        lambda row: f"{row['best_score']:.4f} ± ({row['score_std']:.4f})", axis=1
    )
    for metric in metric_cols:
        result_df[metric] = result_df.apply(
            lambda row: f"{row[metric]:.4f} ± ({row[metric+'_std']:.4f})", axis=1
        )
        # Drop the corresponding std column
        result_df.drop(columns=[metric + "_std"], inplace=True)

    result_df[display_cols].to_csv(rf"{image_path}/{image_folder}/{image_name}.csv")

    return result_df[display_cols]


# Define a custom styling function
def highlight_max_row(s):
    is_max = s == s.max()
    return ["background-color: green" if v else "" for v in is_max]


def highlight_min_row(s):
    is_max = s == s.min()
    return ["background-color: green" if v else "" for v in is_max]

In [13]:
folder = "hyperopt"

# Titanic


In [14]:
display_cols = [
    "model",
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
metric_cols = [
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
dataset_name = "titanic"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, image_folder = folder, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.style.apply(highlight_max_row, subset=metric_cols, axis=0)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png" , dpi=400)
styled_df

,model,accuracy,roc_auc,lift,f1,recall,precision,area_under_pr
0,xgb,0.8361 ± (0.0280),0.8858 ± (0.0275),2.4829 ± (0.1192),0.7703 ± (0.0444),0.7191 ± (0.0522),0.8304 ± (0.0382),0.8640 ± (0.0286)
1,node,0.8216 ± (0.0262),0.8817 ± (0.0219),2.5442 ± (0.0816),0.7704 ± (0.0349),0.7806 ± (0.0460),0.7626 ± (0.0452),0.8550 ± (0.0207)
2,fttransformer,0.8170 ± (0.0202),0.8788 ± (0.0245),2.5442 ± (0.0816),0.7606 ± (0.0313),0.7600 ± (0.0490),0.7630 ± (0.0278),0.8567 ± (0.0240)
3,categoryembedding,0.8114 ± (0.0182),0.8784 ± (0.0270),2.5749 ± (0.0682),0.7567 ± (0.0266),0.7659 ± (0.0414),0.7489 ± (0.0258),0.8584 ± (0.0227)
4,gandalf,0.8092 ± (0.0253),0.8769 ± (0.0224),2.4526 ± (0.1084),0.7545 ± (0.0359),0.7658 ± (0.0466),0.7442 ± (0.0318),0.8486 ± (0.0235)
5,tabnet,0.7923 ± (0.0169),0.8687 ± (0.0223),2.5445 ± (0.0897),0.7296 ± (0.0206),0.7338 ± (0.0836),0.7390 ± (0.0664),0.8388 ± (0.0130)
6,gate,0.8114 ± (0.0201),0.8682 ± (0.0277),2.4829 ± (0.0687),0.7557 ± (0.0318),0.7629 ± (0.0462),0.7493 ± (0.0188),0.8400 ± (0.0304)
7,autoint,0.7968 ± (0.0136),0.8672 ± (0.0194),2.5442 ± (0.0816),0.7476 ± (0.0140),0.7835 ± (0.0244),0.7160 ± (0.0275),0.8436 ± (0.0226)
8,s1dcnn,0.8137 ± (0.0310),0.8595 ± (0.0228),2.4829 ± (0.0687),0.7347 ± (0.0445),0.6725 ± (0.0479),0.8124 ± (0.0602),0.8297 ± (0.0255)
9,mlp,0.8238 ± (0.0282),0.8590 ± (0.0044),2.5133 ± (0.0712),0.7525 ± (0.0335),0.6959 ± (0.0371),0.8245 ± (0.0710),0.8413 ± (0.0205)


# Housing

In [15]:
display_cols = ["model", "mse", "r2_score", "rmse"]
metric_cols = ["mse", "r2_score", "rmse"]
dataset_name = "housing"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, image_folder = folder, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.style.apply(highlight_max_row, subset=["r2_score"], axis=0).apply(
    highlight_min_row, subset=["mse", "rmse"], axis=0
)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png", dpi=400)
styled_df

,model,mse,r2_score,rmse
0,xgb,0.1870 ± (0.0080),0.8595 ± (0.0069),0.4323 ± (0.0093)
1,node,0.3122 ± (0.0223),0.7656 ± (0.0146),0.5584 ± (0.0199)
2,categoryembedding,0.5167 ± (0.0198),0.6768 ± (0.0129),0.7187 ± (0.0139)


# Heloc

In [16]:
display_cols = [
    "model",
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
metric_cols = [
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
dataset_name = "heloc"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, image_folder = folder, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.style.apply(highlight_max_row, subset=metric_cols, axis=0)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png", dpi=400)
styled_df

,model,accuracy,roc_auc,lift,f1,recall,precision,area_under_pr
0,node,0.7183 ± (0.0049),0.7977 ± (0.0042),1.7326 ± (0.0347),0.7224 ± (0.0079),0.7025 ± (0.0166),0.7437 ± (0.0049),0.7968 ± (0.0068)
1,xgb,0.7186 ± (0.0047),0.7929 ± (0.0056),1.7197 ± (0.0508),0.7411 ± (0.0040),0.7716 ± (0.0115),0.7131 ± (0.0078),0.7905 ± (0.0093)
2,categoryembedding,0.7133 ± (0.0064),0.7882 ± (0.0044),1.7124 ± (0.0236),0.7185 ± (0.0140),0.7025 ± (0.0359),0.7369 ± (0.0131),0.7864 ± (0.0056)


# Diabetes

In [17]:
display_cols = [
    "model",
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
metric_cols = [
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
dataset_name = "diabetes"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, image_folder = folder, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.style.apply(highlight_max_row, subset=metric_cols, axis=0)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png", dpi=400)
styled_df

,model,accuracy,roc_auc,lift,f1,recall,precision,area_under_pr
0,xgb,0.8888 ± (0.0002),0.6817 ± (0.0055),2.4958 ± (0.0868),0.0326 ± (0.0095),0.0168 ± (0.0051),0.5628 ± (0.0141),0.2341 ± (0.0081)
1,categoryembedding,0.6358 ± (0.0128),0.6789 ± (0.0056),2.4614 ± (0.0816),0.2741 ± (0.0036),0.6161 ± (0.0181),0.1764 ± (0.0034),0.2275 ± (0.0083)
2,node,0.6364 ± (0.0280),0.6779 ± (0.0060),2.4271 ± (0.0728),0.2726 ± (0.0069),0.6097 ± (0.0357),0.1759 ± (0.0070),0.2199 ± (0.0090)
3,autoint,0.6356 ± (0.0136),0.6754 ± (0.0078),2.3945 ± (0.1032),0.2714 ± (0.0047),0.6080 ± (0.0225),0.1747 ± (0.0036),0.2177 ± (0.0117)


# Creditcard

In [18]:
display_cols = [
    "model",
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
metric_cols = [
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
dataset_name = "creditcard"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, image_folder = folder, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.style.apply(highlight_max_row, subset=metric_cols, axis=0)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png", dpi=400)
styled_df

,model,accuracy,roc_auc,lift,f1,recall,precision,area_under_pr
0,categoryembedding,0.9682 ± (0.0105),0.9861 ± (0.0061),9.6954 ± (0.1288),0.0984 ± (0.0255),0.9207 ± (0.0238),0.0522 ± (0.0142),0.7362 ± (0.0433)
1,xgb,0.9996 ± (0.0001),0.9828 ± (0.0082),9.6547 ± (0.2193),0.8584 ± (0.0211),0.7886 ± (0.0216),0.9420 ± (0.0262),0.8548 ± (0.0246)
2,autoint,0.9722 ± (0.0092),0.9851 ± (0.0056),9.5531 ± (0.1891),0.1109 ± (0.0321),0.9086 ± (0.0346),0.0595 ± (0.0185),0.7439 ± (0.0173)
3,tabtransformer,0.7547 ± (0.2681),0.9330 ± (0.0634),8.9813 ± (0.7994),0.0749 ± (0.0677),0.8922 ± (0.0347),0.0405 ± (0.0376),0.6181 ± (0.0664)


# Adult

In [19]:
display_cols = [
    "model",
    "accuracy",
    "roc_auc",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
metric_cols = ["accuracy", "roc_auc", "f1", "recall", "precision", "area_under_pr"]
dataset_name = "adult"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, image_folder = folder, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.style.apply(highlight_max_row, subset=metric_cols, axis=0)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png", dpi=400)
styled_df

,model,accuracy,roc_auc,f1,recall,precision,area_under_pr
0,xgb,0.8737 ± (0.0034),0.9286 ± (0.0013),0.7130 ± (0.0079),0.6516 ± (0.0100),0.7873 ± (0.0100),0.8296 ± (0.0037)
1,node,0.8137 ± (0.0046),0.9157 ± (0.0019),0.6911 ± (0.0049),0.8649 ± (0.0076),0.5755 ± (0.0075),0.7911 ± (0.0064)


# Iris

In [20]:
display_cols = ["model", "accuracy", "f1"]
metric_cols = [
    "accuracy",
    "f1",
]
dataset_name = "iris"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, image_folder = folder, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.style.apply(highlight_max_row, subset=metric_cols, axis=0)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png", dpi=400)
styled_df

,model,accuracy,f1
0,autoint,0.9867 ± (0.0163),0.9866 ± (0.0164)
1,fttransformer,0.9800 ± (0.0267),0.9798 ± (0.0269)
2,gandalf,0.9800 ± (0.0267),0.9800 ± (0.0267)
3,node,0.9800 ± (0.0267),0.9800 ± (0.0267)
4,tabtransformer,0.9800 ± (0.0400),0.9799 ± (0.0401)
5,categoryembedding,0.9733 ± (0.0249),0.9733 ± (0.0250)
6,tabnet,0.9733 ± (0.0327),0.9732 ± (0.0328)
7,s1dcnn,0.9733 ± (0.0389),0.9733 ± (0.0390)
8,gate,0.9667 ± (0.0422),0.9666 ± (0.0422)
9,mlp,0.9667 ± (0.0422),0.9665 ± (0.0423)


# Covertype

In [21]:
display_cols = ["model", "accuracy", "f1"]
metric_cols = ["accuracy", "f1"]
dataset_name = "covertype"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, image_folder = folder, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.style.apply(highlight_max_row, subset=metric_cols, axis=0)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png", dpi=400)
styled_df

,model,accuracy,f1
0,categoryembedding,0.8634 ± (0.0093),0.8670 ± (0.0087)


# Breastcancer

In [22]:
display_cols = [
    "model",
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
metric_cols = [
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
dataset_name = "breastcancer"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, image_folder = folder, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.sort_values("f1", ascending=False).style.apply(
    highlight_max_row, subset=metric_cols, axis=0
)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png", dpi=400)
styled_df

,model,accuracy,roc_auc,lift,f1,recall,precision,area_under_pr
0,gate,0.9895 ± (0.0102),0.9962 ± (0.0075),1.5939 ± (0.0100),0.9917 ± (0.0081),0.9972 ± (0.0056),0.9863 ± (0.0123),0.9973 ± (0.0055)
1,node,0.9859 ± (0.0119),0.9943 ± (0.0063),1.5911 ± (0.0121),0.9889 ± (0.0093),0.9944 ± (0.0068),0.9836 ± (0.0158),0.9959 ± (0.0049)
2,gandalf,0.9859 ± (0.0070),0.9958 ± (0.0061),1.5939 ± (0.0100),0.9889 ± (0.0054),0.9944 ± (0.0069),0.9837 ± (0.0157),0.9971 ± (0.0045)
4,categoryembedding,0.9859 ± (0.0070),0.9968 ± (0.0037),1.5939 ± (0.0100),0.9888 ± (0.0056),0.9916 ± (0.0069),0.9863 ± (0.0123),0.9979 ± (0.0026)
3,fttransformer,0.9859 ± (0.0070),0.9964 ± (0.0060),1.5939 ± (0.0100),0.9888 ± (0.0055),0.9916 ± (0.0068),0.9861 ± (0.0087),0.9974 ± (0.0044)
5,tabtransformer,0.9842 ± (0.0066),0.9967 ± (0.0049),1.5939 ± (0.0100),0.9875 ± (0.0051),0.9944 ± (0.0069),0.9808 ± (0.0104),0.9977 ± (0.0035)
6,mlp,0.9807 ± (0.0116),0.9966 ± (0.0046),1.5939 ± (0.0100),0.9847 ± (0.0092),0.9916 ± (0.0112),0.9781 ± (0.0138),0.9977 ± (0.0032)
8,s1dcnn,0.9807 ± (0.0066),0.9962 ± (0.0048),1.5939 ± (0.0100),0.9846 ± (0.0053),0.9860 ± (0.0154),0.9837 ± (0.0157),0.9974 ± (0.0034)
7,autoint,0.9807 ± (0.0065),0.9947 ± (0.0055),1.5939 ± (0.0100),0.9846 ± (0.0052),0.9860 ± (0.0088),0.9834 ± (0.0103),0.9965 ± (0.0039)
9,xgb,0.9631 ± (0.0129),0.9909 ± (0.0066),1.5939 ± (0.0100),0.9708 ± (0.0099),0.9748 ± (0.0106),0.9673 ± (0.0235),0.9939 ± (0.0046)


# Ageconditions

In [24]:
display_cols = [
    "model",
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
metric_cols = [
    "accuracy",
    "roc_auc",
    "lift",
    "f1",
    "recall",
    "precision",
    "area_under_pr",
]
dataset_name = "ageconditions"
df = make_results_table(
    best_df, dataset_name, display_cols, metric_cols, image_name=dataset_name, image_folder = folder, dpi=1200
)
# Apply the styling function to the specified columns
styled_df = df.sort_values("f1", ascending=False).style.apply(
    highlight_max_row, subset=metric_cols, axis=0
)
dfi.export(styled_df, f"./tables/{folder}/df_styled_{dataset_name}.png", dpi=400)
styled_df

,model,accuracy,roc_auc,lift,f1,recall,precision,area_under_pr
0,node,0.9196 ± (0.0368),0.9559 ± (0.0277),5.1878 ± (0.4067),0.7967 ± (0.0933),0.8883 ± (0.0871),0.7270 ± (0.1176),0.8587 ± (0.0827)
1,gandalf,0.9116 ± (0.0264),0.9419 ± (0.0184),5.0939 ± (0.5412),0.7706 ± (0.0628),0.8429 ± (0.0363),0.7115 ± (0.0828),0.8143 ± (0.0692)
2,xgb,0.9261 ± (0.0170),0.9537 ± (0.0294),5.2773 ± (0.2383),0.7588 ± (0.0599),0.6749 ± (0.0701),0.8710 ± (0.0616),0.8675 ± (0.0575)
3,s1dcnn,0.9051 ± (0.0251),0.9270 ± (0.0321),4.6122 ± (0.4308),0.7316 ± (0.0674),0.7411 ± (0.0606),0.7228 ± (0.0752),0.7287 ± (0.0970)
4,mlp,0.9131 ± (0.0237),0.9237 ± (0.0155),5.0902 ± (0.6648),0.7269 ± (0.0830),0.6771 ± (0.1094),0.7952 ± (0.0684),0.8040 ± (0.0767)
5,autoint,0.8940 ± (0.0282),0.9157 ± (0.0306),4.6167 ± (0.4754),0.7251 ± (0.0657),0.7970 ± (0.0599),0.6680 ± (0.0810),0.7525 ± (0.0583)
6,categoryembedding,0.8892 ± (0.0353),0.9174 ± (0.0410),4.6204 ± (0.7371),0.7234 ± (0.0604),0.8156 ± (0.0562),0.6579 ± (0.0909),0.7434 ± (0.0915)
7,gate,0.8976 ± (0.0436),0.9260 ± (0.0476),4.8250 ± (0.4475),0.7233 ± (0.1096),0.7684 ± (0.1305),0.6938 ± (0.1199),0.8063 ± (0.0836)
8,fttransformer,0.8908 ± (0.0340),0.9151 ± (0.0642),4.8098 ± (0.7421),0.7205 ± (0.0842),0.8074 ± (0.1075),0.6545 ± (0.0833),0.7900 ± (0.1006)
9,tabtransformer,0.8778 ± (0.0117),0.8951 ± (0.0280),4.3222 ± (0.4471),0.7009 ± (0.0260),0.8242 ± (0.0333),0.6100 ± (0.0247),0.6997 ± (0.0805)
